In [221]:
import json
import pandas as pd
  
df = pd.read_json("/Users/oceane/Desktop/plan-your-trip-with-kayak/coordinates.json")

df[['longitude','latitude']] = df.coordinates.apply(lambda x: pd.Series(str(x).split(",")))
df = df.drop(columns=['coordinates'])


In [222]:
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)

In [223]:
import requests
weather = []
for index, row in df.iterrows():
    appid = '7e208a67c297443752c742b0cdd73fdf'
    lat= row['latitude']
    lon= row['longitude']
    city = row['city']
    # URL = 'https://api.openweathermap.org/data/3.0/onecall'
    # PARAMS = {'lat':lat, 'lon':lon, 'appid': appid, 'units': 'metrics', 'cnt':7, 'exclude':'urrent,minutely,hourly,alerts'}
    # r = requests.get(url = URL, params=PARAMS).json()
    URL = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&exclude=current,minutely,hourly,alerts&appid={appid}'
    r = requests.get(URL).json()
    for i in range(7):
        weather.append({'city':city,'lat':lat, 'lon':lon, 'day': r['daily'][i]['dt'], 'temp':r['daily'][i]['temp']['day']})

    

In [224]:

with open('weather.json', 'w') as f:
    f.write(json.dumps(weather, indent=2))


In [225]:
weather_data = pd.read_json("weather.json")
weather_data.head(7)

,city,lat,lon,day,temp
0,Mont Saint-Michel,48.635954,-1.51146,1667214000,15.09
1,Mont Saint-Michel,48.635954,-1.51146,1667300400,16.46
2,Mont Saint-Michel,48.635954,-1.51146,1667386800,14.21
3,Mont Saint-Michel,48.635954,-1.51146,1667473200,12.18
4,Mont Saint-Michel,48.635954,-1.51146,1667559600,12.86
5,Mont Saint-Michel,48.635954,-1.51146,1667646000,14.22
6,Mont Saint-Michel,48.635954,-1.51146,1667732400,15.96


In [226]:
weather_data['temp_mean'] = weather_data['temp'].groupby(weather_data['lat']).transform('mean')



In [227]:
weather_data = weather_data.drop(columns=['temp'])
weather_data = weather_data.drop(columns=['day'])
weather_data.round({'temp_mean': 1})

,city,lat,lon,temp_mean
0,Mont Saint-Michel,48.635954,-1.511460,14.4
1,Mont Saint-Michel,48.635954,-1.511460,14.4
2,Mont Saint-Michel,48.635954,-1.511460,14.4
3,Mont Saint-Michel,48.635954,-1.511460,14.4
4,Mont Saint-Michel,48.635954,-1.511460,14.4
...,...,...,...,...
240,La Rochelle,46.159113,-1.152043,16.8
241,La Rochelle,46.159113,-1.152043,16.8
242,La Rochelle,46.159113,-1.152043,16.8
243,La Rochelle,46.159113,-1.152043,16.8


In [228]:
weather_data = weather_data.drop_duplicates(subset=['lat', 'lon'])
weather_data = weather_data.reset_index(drop=True)

In [229]:
weather_data

,city,lat,lon,temp_mean
0,Mont Saint-Michel,48.635954,-1.511460,14.425714
1,Saint-Malo,48.649518,-2.026041,14.651429
2,Bayeux,49.276462,-0.702474,14.140000
3,Le Havre,49.493898,0.107973,14.177143
4,Rouen,49.440459,1.093966,13.644286
5,Paris,48.858890,2.320041,14.772857
6,Amiens,49.894171,2.295695,13.610000
7,Lille,50.636565,3.063528,13.112857
8,Strasbourg,48.584614,7.750713,13.605714
9,Ch\u00e2teau du Haut-K\u0153nigsbourg,48.249490,7.344296,10.611429


In [234]:
weather_data = weather_data.sort_values(by='temp_mean', ascending=False)

In [235]:
top_five_weather = weather_data.head(5)

In [236]:
top_five_weather.round({'temp_mean': 1})

,city,lat,lon,temp_mean
27,Collioure,42.525050,3.083155,19.8
19,Cassis,43.214036,5.539632,19.0
20,Marseille,43.296174,5.369953,18.9
18,Bormes-les-Mimosas,43.150697,6.341928,18.8
25,Aigues-Mortes,43.565823,4.191284,18.8


In [238]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter_mapbox(top_five_weather, lat="lat", lon="lon", color="temp_mean", size="temp_mean", hover_name="city", zoom=6, mapbox_style="carto-positron")
fig.show()